# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [1]:
import os
import shutil
import numpy as np
#from tqdm.notebook import tqdm
from tqdm import tqdm

import binarybrain as bb

import torch
import torchvision
import torchvision.transforms as transforms

device_id = 0
bb.set_device(device_id)
print(f"BinaryBrain : {bb.get_version_string()}")
print(f"Device : {bb.get_device_name(device_id)}")

BinaryBrain : 4.3.2
Device : NVIDIA GeForce GTX 1660 SUPER


異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [2]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLut4Simple'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist/tb_mnist_lut_simple'
rtl_module_name       = 'MnistLutSimple'
output_velilog_file   = os.path.join(data_path, rtl_module_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 8
mini_batch_size       = 64
frame_modulation_size = 15

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [3]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [4]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
            bb.DifferentiableLut([64*4*2], connection='random', N=4),
            bb.DifferentiableLut([64*4],   connection='serial', N=4),
            bb.DifferentiableLut([64],     connection='serial', N=4),
            bb.DifferentiableLut([10*4*4], connection='random', N=4),
            bb.DifferentiableLut([10*4],   connection='serial', N=4),
            bb.AverageLut       ([10],     connection='serial', N=4),
            bb.BinaryToReal(frame_integration_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam(learning_rate=0.001)

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [ ]:
#bb.load_networks(data_path, net)

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(images.numpy().astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[labels.numpy()].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(images.numpy().astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[labels.numpy()].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(data_path, net)

  0%|          | 0/938 [00:00<?, ?it/s]/tmp/ipykernel_23608/990070199.py:10: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
/tmp/ipykernel_23608/990070199.py:11: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
100%|██████████| 938/938 [00:03<00:00, 256.66it/s, acc=0.733, loss=1.71]


epoch[0] : loss=1.696060 accuracy=0.757300


100%|██████████| 938/938 [00:04<00:00, 229.71it/s, acc=0.778, loss=1.68]


epoch[1] : loss=1.695728 accuracy=0.749400


100%|██████████| 938/938 [00:04<00:00, 231.00it/s, acc=0.783, loss=1.67]


...

epoch[5] : loss=1.685567 accuracy=0.783400


100%|██████████| 938/938 [00:04<00:00, 227.34it/s, acc=0.795, loss=1.67]


epoch[6] : loss=1.670720 accuracy=0.798500


100%|██████████| 938/938 [00:03<00:00, 239.91it/s, acc=0.798, loss=1.67]


epoch[7] : loss=1.666948 accuracy=0.786400


## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [6]:
# export verilog
bb.load_networks(data_path, net)

# 結果を出力
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_layers(f, module_name=rtl_module_name, net=net, device="")

# Simulation用ファイルに上書きコピー
os.makedirs(rtl_sim_path, exist_ok=True)
shutil.copyfile(output_velilog_file, sim_velilog_file)

'../../verilog/mnist/tb_mnist_lut_simple/MnistLutSimple.v'

In [7]:
# シミュレーション用データファイル作成
with open(os.path.join(rtl_sim_path, 'mnist_test.txt'), 'w') as f:
    bb.dump_verilog_readmemb_image_classification(f ,loader_test)

## モデルの内部の値を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [8]:
# 最新の保存データ読み込み
bb.load_networks(data_path, net)

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、4つの入力と接続されており、layer0 の出力ノードは 256 個あるので、256x4 の行列が取得できます。

In [9]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(512, 4)


array([[531, 302, 316, 420],
       [736, 111,  73, 191],
       [529, 203, 400,  20],
       ...,
       [770, 570, 749, 161],
       [633, 361, 153, 380],
       [221, 664, 145, 316]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
4入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [10]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(512, 16)


array([[False, False, False, ...,  True,  True, False],
       [False, False, False, ...,  True,  True,  True],
       [ True, False, False, ..., False, False, False],
       ...,
       [False, False,  True, ..., False,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

### 重み行列を覗いてみる

4入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [11]:
W = layer0.W().numpy()
print(W.shape)
W

(512, 16)


array([[0.48839268, 0.46542346, 0.4164946 , ..., 0.577624  , 0.5217293 ,
        0.50701636],
       [0.48557302, 0.34633306, 0.4835357 , ..., 0.49888486, 0.5800752 ,
        0.50812733],
       [0.6114968 , 0.46000057, 0.42215806, ..., 0.50530773, 0.4434847 ,
        0.36112332],
       ...,
       [0.45254043, 0.3687159 , 0.5902619 , ..., 0.45086637, 0.51333576,
        0.5051012 ],
       [0.47382638, 0.55758375, 0.51282316, ..., 0.44694746, 0.49972624,
        0.46004656],
       [0.48367164, 0.5448981 , 0.5431081 , ..., 0.5861375 , 0.4651559 ,
        0.54050434]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [12]:
# そのままだとすべて0なので、1回だけbackward実施
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

dW = layer0.dW().numpy()
print(dW.shape)
dW

(512, 16)


/tmp/ipykernel_23608/4074597919.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
/tmp/ipykernel_23608/4074597919.py:4: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))


array([[ 2.60687259e-04,  8.68957577e-05, -9.74802242e-05, ...,
        -1.73727130e-06, -4.91904066e-05, -1.63967998e-05],
       [ 1.09896064e-07,  3.58559191e-08,  3.58559191e-08, ...,
        -1.22236088e-08, -1.22236088e-08, -3.89991328e-09],
       [-1.17297843e-03, -3.93184600e-04, -6.30546012e-04, ...,
         1.64355064e-04,  1.10299239e-04,  3.01924883e-05],
       ...,
       [-2.87242699e-04, -2.54883198e-05, -2.48015800e-04, ...,
         3.39632243e-05,  6.87251450e-05,  2.55105588e-05],
       [-4.50787041e-03, -5.04939584e-04, -1.44079211e-03, ...,
         7.17333052e-04,  3.79461941e-04,  4.24605096e-04],
       [-2.03480013e-05, -6.78293873e-06, -5.75860031e-06, ...,
         3.53198266e-06, -1.88918784e-06, -6.29708666e-07]], dtype=float32)